# API Connectivity Test

Quick 3-line tests for all API keys in `.env` file.

**APIs:** OpenAI, Massive, Alpha Vantage, FMP, Finnhub

In [2]:
# Setup
import os, requests, time
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Path().resolve().parent / '.env', override=True)

def test_api(name, key_var, test_func):
    key = os.getenv(key_var)
    if not key: return print(f"{name}: ⚠ SKIPPED (no {key_var})")
    try:
        result = test_func(key)
        print(f"{name}: ✅ SUCCESS - {result}")
    except Exception as e:
        print(f"{name}: ❌ FAILED - {str(e)[:100]}")

## 1. OpenAI

In [3]:
from openai import OpenAI
test_api("OpenAI", "OPENAI_API_KEY", lambda k: OpenAI(api_key=k, base_url=os.getenv('OPENAI_API_URL')).chat.completions.create(model=os.getenv('OPENAI_MODEL', 'gpt-4o-mini'), messages=[{"role": "user", "content": "Say OK"}], max_tokens=5).choices[0].message.content)

OpenAI: ✅ SUCCESS - OK!


## 2. Massive (S3)

In [4]:
import boto3
from botocore.config import Config
test_api("Massive S3", "MASSIVE_S3_ACCESS_KEY", lambda k: boto3.Session(aws_access_key_id=k, aws_secret_access_key=os.getenv('MASSIVE_S3_SECRET_KEY')).client('s3', endpoint_url=os.getenv('MASSIVE_S3_ENDPOINT', 'https://files.massive.com'), config=Config(signature_version='s3v4')).list_objects_v2(Bucket=os.getenv('MASSIVE_S3_BUCKET', 'flatfiles'), Prefix='us_stocks_sip', MaxKeys=1)['KeyCount'])

Massive S3: ⚠ SKIPPED (no MASSIVE_S3_ACCESS_KEY)


## 3. Alpha Vantage

In [5]:
test_api("Alpha Vantage", "ALPHA_VANTAGE_API_KEY", lambda k: requests.get('https://www.alphavantage.co/query', params={'function': 'TIME_SERIES_INTRADAY', 'symbol': 'AAPL', 'interval': '1min', 'apikey': k}, timeout=10).json()['Meta Data']['2. Symbol'])

Alpha Vantage: ❌ FAILED - 'Meta Data'


## 4. FMP (Financial Modeling Prep)

In [6]:
test_api("FMP", "FMP_API_KEY", lambda k: requests.get('https://financialmodelingprep.com/api/v3/profile/AAPL', params={'apikey': k}, timeout=10).json()[0]['companyName'])

FMP: ❌ FAILED - 0


## 5. Finnhub

In [7]:
test_api("Finnhub", "FINNHUB_API_KEY", lambda k: requests.get('https://finnhub.io/api/v1/quote', params={'symbol': 'AAPL', 'token': k}, timeout=10).json()['c'])

Finnhub: ✅ SUCCESS - 255.53
